In [1]:
import bugzilla
import requests
import time
from datetime import datetime

In [2]:
pagure_owner_alias = requests.get("https://src.fedoraproject.org/extras/pagure_owner_alias.json").json()["rpms"]

In [3]:
orphaned = [p for p in pagure_owner_alias if "orphan" in pagure_owner_alias[p]]
orphaned[:10]

['7kaa',
 '8Kingdoms',
 '915resolution',
 '99soft-oss-parent',
 'AcetoneISO2',
 'Ajaxterm',
 'CableSwig',
 'Canna',
 'CastPodder',
 'CodeAnalyst-gui']

In [4]:
bzapi = bugzilla.Bugzilla("bugzilla.redhat.com")

In [5]:
if not bzapi.logged_in:
    bzapi.interactive_login()

In [6]:
TRACKER = 1700317

query = bzapi.build_query(product="Fedora", include_fields=["flags", "assigned_to", "component", "creation_time"])
query["blocks"] = TRACKER
query["status"] = "NEW"
bugzillas = bzapi.query(query)
bugzillas[:10]

[<Bug #1701227 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f2501296790>,
 <Bug #1714212 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f2501296e10>,
 <Bug #1716412 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f2501296dd0>,
 <Bug #1716488 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f2501293350>,
 <Bug #1716525 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f25012932d0>,
 <Bug #1716546 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f250128df50>,
 <Bug #1729908 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f250128de90>,
 <Bug #1734875 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f250128dc10>,
 <Bug #1734882 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f250128dd50>,
 <Bug #1734884 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f250128df90>]

In [7]:
RELENG = "releng@fedoraproject.org"
ORPHAN = "extras-orphan@fedoraproject.org"

In [8]:
pkgs_to_orphan = []
bugs_to_reassign = []

In [9]:
for bug in bugzillas:
    if bug.component in orphaned:
        if bug.assigned_to != ORPHAN:
            bugs_to_reassign.append(bug.id)
        continue
    needinfos = bug.get_flags("needinfo")
    for needinfo in needinfos:
        if needinfo['requestee'] == bug.assigned_to:
            break
    else:
        # no needinfo on assignee
        continue
    bug_state = 0
    last_time = datetime.fromtimestamp(time.mktime(bug.creation_time.timetuple()))
    for comment in bug.getcomments():
        if comment["creator"] != RELENG:
            continue
        if not comment["text"].startswith("Dear Maintainer,"):
            continue
        comment_time = datetime.fromtimestamp(time.mktime(comment["creation_time"].timetuple()))
        delta = comment_time - last_time
        if bug_state == 0 and delta.days >= 7-1:
            last_time = comment_time
            bug_state = 1
        elif bug_state == 1 and delta.days >= 3*7-1:
            last_time = comment_time
            bug_state = 2
            break
    delta = datetime.now() - last_time
    if bug_state == 2 and delta.days >= 4*7-1:
        to_orphan = f"rpms/{bug.component}"
        pkgs_to_orphan.append(to_orphan)
        print(to_orphan)
        bugs_to_reassign.append(bug.id)

rpms/ferm
rpms/python-alchimia
rpms/vdirsyncer
rpms/python-sanic
rpms/ExchangeIR
rpms/aespipe
rpms/apt-cacher-ng
rpms/archaius
rpms/atanua
rpms/busybox
rpms/bval
rpms/camotics
rpms/cduce
rpms/clapham
rpms/couchdb
rpms/csstidy
rpms/delve
rpms/dumb-init
rpms/eclipse-anyedit
rpms/eclipse-avr
rpms/eclipse-checkstyle
rpms/eclipse-color-theme
rpms/eclipse-dltk
rpms/eclipse-egit
rpms/eclipse-emf
rpms/eclipse-epic
rpms/eclipse-gef
rpms/eclipse-license
rpms/eclipse-m2e-antlr
rpms/eclipse-m2e-apt
rpms/eclipse-m2e-buildhelper
rpms/eclipse-m2e-core
rpms/eclipse-m2e-cxf
rpms/eclipse-m2e-egit
rpms/eclipse-m2e-maven-dependency-plugin
rpms/eclipse-m2e-mavenarchiver
rpms/eclipse-m2e-mavendev
rpms/eclipse-m2e-modello
rpms/eclipse-m2e-plexus
rpms/eclipse-m2e-sisu
rpms/eclipse-m2e-takari
rpms/eclipse-m2e-tycho
rpms/eclipse-m2e-workspace
rpms/eclipse-m2e-wtp
rpms/eclipse-nls
rpms/eclipse-pdt
rpms/eclipse-photran
rpms/eclipse-ptp
rpms/eclipse-quickrex
rpms/eclipse-remote
rpms/eclipse-sgx
rpms/eclipse-subcli

In [10]:
print("https://bugzilla.redhat.com/buglist.cgi?bug_id=" + ",".join(str(b) for b in bugs_to_reassign))

https://bugzilla.redhat.com/buglist.cgi?bug_id=1701227,1716412,1716488,1716525,1716546,1729908,1734882,1734927,1734938,1734940,1734950,1734993,1734994,1734998,1735008,1735018,1735052,1735057,1735058,1735071,1735104,1735112,1735113,1735115,1735116,1735117,1735118,1735122,1735124,1735125,1735127,1735130,1735132,1735133,1735134,1735135,1735136,1735137,1735138,1735139,1735140,1735142,1735143,1735144,1735145,1735146,1735147,1735148,1735152,1735155,1735156,1735157,1735159,1735160,1735161,1735162,1735164,1735166,1735167,1735168,1735184,1735185,1735186,1735187,1735188,1735189,1735191,1735193,1735216,1735250,1735257,1735267,1735270,1735282,1735310,1735311,1735327,1735328,1735414,1735456,1735458,1735469,1735800,1735803,1735821,1735855,1735884,1735890,1735921,1735930,1735931,1735932,1735933,1735934,1735935,1735936,1735937,1735938,1735939,1735940,1735942,1735943,1735944,1735945,1735946,1735947,1735948,1735949,1735950,1735951,1735952,1735953,1735954,1735955,1735956,1735957,1735958,1735959,1735960,1